In [3]:
import psycopg2
import config
from configparser import ConfigParser

### Connect to the PostgreSQL Database

In [15]:
def config(filename='./Database/database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [17]:
# Connect to the PostgreSQL database server
conn = None
try:
    # read connection parameters
    params = config()

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)
    conn.autocommit = True

    # create a cursor
    cur = conn.cursor()
    print('Connected')

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
Connected


In [ ]:
# This will create a new schema. The schema in PostgreSQL behaves in the same way as databases in other
# relational database systems
database = (
    """DROP SCHEMA IF EXISTS gdelt CASCADE;""",
    """CREATE SCHEMA gdelt;"""
)

# Execute the two database statements
for data in database:
    cur.execute(data)
    
# Commit the executed commands to the database (MUST BE PERFORMED!)    
conn.commit()

## Drop Existing Tables

We are simply removing all of the tables to avoid any errors in building them in the next step.

In [ ]:
drops = (
    """DROP TABLE IF EXISTS gdelt.event;""",
    """DROP TABLE IF EXISTS gdelt.mention;""",
    """DROP TABLE IF EXISTS gdelt.domestic_generic_codes;""",
    """DROP TABLE IF EXISTS gdelt.ethnic_group_codes;""",
    """DROP TABLE IF EXISTS gdelt.event_base_codes;""",
    """DROP TABLE IF EXISTS gdelt.event_codes;""",
    """DROP TABLE IF EXISTS gdelt.event_root_codes;""",
    """DROP TABLE IF EXISTS gdelt.international_actor_special_codes;""",
    """DROP TABLE IF EXISTS gdelt.international_actor_generic_codes;""",
    """DROP TABLE IF EXISTS gdelt.international_region_codes;""",
    """DROP TABLE IF EXISTS gdelt.keds_project_actor_codes;""",
    """DROP TABLE IF EXISTS gdelt.main_ethnic_groups_in_keds_regional_dictionaries;""",
    """DROP TABLE IF EXISTS gdelt.religious_classification_system;"""
)

In [ ]:
# Execute the DROP commands
for drop in drops:
    cur.execute(drop)
    
# Commit the executed commands to the database (MUST BE PERFORMED!)    
conn.commit()

## Build the Database Tables

For each of the tables we will run one command to drop the table if it exists and then another command to build the table from scratch.

### Event Table

In [ ]:
event = (
    
    """
    CREATE TABLE gdelt.event (
        GlobalEventID, 
        Day VARCHAR(), 
        MonthYear VARCHAR(), 
        Year VARCHAR(), 
        FractionDate VARCHAR(), 
        Actor1Code VARCHAR(), 
        Actor1Name VARCHAR(), 
        Actor1CountryCode VARCHAR(), 
        Actor1KnownGroupCode VARCHAR(), 
        Actor1EthnicCode VARCHAR(), 
        Actor1Religion1Code VARCHAR(), 
        Actor1Religion2Code VARCHAR(), 
        Actor1Type1Code VARCHAR(), 
        Actor1Type2Code VARCHAR(), 
        Actor1Type3Code VARCHAR(), 
        Actor2Code VARCHAR(), 
        Actor2Name VARCHAR(), 
        Actor2CountryCode VARCHAR(), 
        Actor2KnownGroupCode VARCHAR(), 
        Actor2EthnicCode VARCHAR(), 
        Actor2Religion1Code VARCHAR(), 
        Actor2Religion2Code VARCHAR(), 
        Actor2Type1Code VARCHAR(), 
        Actor2Type2Code VARCHAR(), 
        Actor2Type3Code VARCHAR(), 
        IsRootEvent VARCHAR(), 
        EventCode VARCHAR(), 
        EventBaseCode VARCHAR(), 
        EventRootCode VARCHAR(), 
        QuadClass VARCHAR(), 
        GoldsteinScale VARCHAR(), 
        NumMentions VARCHAR(), 
        NumSources VARCHAR(), 
        NumArticles VARCHAR(), 
        AvgTone VARCHAR(), 
        Actor1Geo_Type VARCHAR(), 
        Actor1Geo_Fullname VARCHAR(), 
        Actor1Geo_CountryCode VARCHAR(), 
        Actor1Geo_ADM1Code VARCHAR(), 
        Actor1Geo_ADM2Code VARCHAR(), 
        Actor1Geo_Lat VARCHAR(), 
        Actor1Geo_Long VARCHAR(), 
        Actor1Geo_FeatureID VARCHAR(), 
        Actor2Geo_Type VARCHAR(), 
        Actor2Geo_Fullname VARCHAR(), 
        Actor2Geo_CountryCode VARCHAR(), 
        Actor2Geo_ADM1Code VARCHAR(), 
        Actor2Geo_ADM2Code VARCHAR(), 
        Actor2Geo_Lat VARCHAR(), 
        Actor2Geo_Long VARCHAR(), 
        Actor2Geo_FeatureID VARCHAR(), 
        ActionGeo_Type VARCHAR(), 
        ActionGeo_Fullname VARCHAR(), 
        ActionGeo_CountryCode VARCHAR(), 
        ActionGeo_ADM1Code VARCHAR(), 
        ActionGeo_ADM2Code VARCHAR(), 
        ActionGeo_Lat VARCHAR(), 
        ActionGeo_Long VARCHAR(), 
        ActionGeo_FeatureID VARCHAR(), 
        DATEADDED VARCHAR(), 
        SOURCEURL VARCHAR()
    );
    """
)

# Execute the CREATE TABLE command
cur.execute(event)
    
# Commit the executed command to the database (MUST BE PERFORMED!)    
conn.commit()

### Mention Tables

In [ ]:
mention = (

    """
    CREATE TABLE gdelt.mention(
        GlobalEventID VARCHAR(),
        EventTimeDate VARCHAR(),
        MentionTimeDate VARCHAR(),
        MentionType VARCHAR(),
        MentionSourceName VARCHAR(),
        MentionIdentifier VARCHAR(),
        SentenceID VARCHAR(),
        Actor1CharOffset VARCHAR(),
        Actor2CharOffset VARCHAR(),
        ActionCharOffset VARCHAR(),
        InRawText VARCHAR(),
        Confidence VARCHAR(),
        MentionDocLen VARCHAR(),
        MentionDocTone VARCHAR(),
        MentionDocTranslationInfo VARCHAR(),
        Extras VARCHAR()
    );
    """
)

cur.execute(mention)
    
conn.commit()

### Domestic Generic Codes Table

In [ ]:
dgc = (

    """
    CREATE TABLE domestic_generic_codes (
        code VARCHAR(3) NOT NULL,
        code_desc VARCHAR(200),
        role VARCHAR(20),
        PRIMARY KEY (code)
    );
    """
)

cur.execute(dgc)
    
conn.commit()

### Ethnic Group Codes Table

In [ ]:
egc = (

    """
    CREATE TABLE ethnic_group_codes (
        ethnic_group_code VARCHAR(3) NOT NULL,
        ethnic_group VARCHAR(35),
        ethnic_group_countries VARCHAR(145),
        PRIMARY KEY (ethnic_group_code)
    );
    """
)

cur.execute(egc)
    
conn.commit()

### Event Base Codes Table

In [ ]:
ebc = (

    """
    CREATE TABLE event_base_codes (
        EventBaseCode CHAR(3) NOT NULL,
        EventRootCode CHAR(2),
        EventBaseCode_Desc VARCHAR(75),
        PRIMARY KEY (EventBaseCode)
    );
    """
)

cur.execute(ebc)
    
conn.commit()

### Event Codes Table

In [ ]:
eventcodes = (

    """
    CREATE TABLE event_codes (
        EventCode VARCHAR(4) NOT NULL,
        EventBaseCode VARCHAR(3),
        EventRootCode VARCHAR(2),
        EventCode_Desc VARCHAR(75),
        PRIMARY KEY (EventCode)
    );
    """
)

cur.execute(eventcodes)
    
conn.commit()

### Event Root Codes Table

In [ ]:
eventroot = (

    """
    CREATE TABLE event_root_codes (
        EventRootCode VARCHAR(2) NOT NULL,
        EventRootCode_Desc VARCHAR(35),
        PRIMARY KEY (EventRootCode)
    );
    """
)

cur.execute(eventroot)
    
conn.commit()

### International Actor Special Codes Table

In [ ]:
iasc = (

    """
    CREATE TABLE international_actor_special_codes (
        international_actor_special_code VARCHAR(10) NOT NULL,
        international_actor_special VARCHAR(60),
        international_actor_special_region VARCHAR(15),
        PRIMARY KEY (international_actor_special_code)
    );
    """
)

cur.execute(iasc)
    
conn.commit()

### International Actor Generic Codes Table

In [ ]:
iagc = (

    """
    CREATE TABLE international_actor_generic_codes (
        international_actor_generic_code VARCHAR(3) NOT NULL,
        international_actor_generic VARCHAR(100),
        international_actor_generic_example VARCHAR(60),
        international_actor_generic_full_code VARCHAR(25)
        PRIMARY KEY (international_actor_generic_code)
    );
    """
)

cur.execute(iagc)
    
conn.commit()

### International Region Codes Table

In [ ]:
irc = (

    """
    CREATE TABLE international_region_codes (
        region_code VARCHAR(3) NOT NULL,
        region VARCHAR(20)
        PRIMARY KEY (region_code)
    );
    """
)

cur.execute(irc)
    
conn.commit()

### Keds Project Actor Codes Table

In [ ]:
kpac = (

    """
    CREATE TABLE keds_project_actor_codes (
        keds_project_actor_code VARCHAR(15) NOT NULL,
        keds_project_actor VARCHAR(60),
        PRIMARY KEY (keds_project_actor_code)
    );
    """
)

cur.execute(kpac)

conn.commit()

### Main Ethnic Groups in keds Regional Dictionaries Table

In [ ]:
megkrd = (

    """
    CREATE TABLE main_ethnic_groups_in_keds_regional_dictionaries (
        ethnic_group_code VARCHAR(3) NOT NULL,
        ethnic_group VARCHAR(15),
        PRIMARY KEY (ethnic_group_code)
    );
    """
)

cur.execute(megkrd)

conn.commit()

### Religious Classification System Table

In [ ]:
rcst = (

    """
    CREATE TABLE religious_classification_system (
        Hierarchical_Code VARCHAR(20) NOT NULL,
        Religion_and_Comments VARCHAR(120),
        PRIMARY KEY (Hierarchical_Code)
    );
    """
)

cur.execute(rcst)

conn.commit()

### Event Data

In [ ]:
copy_sql = """
           COPY acled.event FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/inputs/event.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Event Type Data

In [ ]:
copy_sql = """
           COPY acled.event_type FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/inputs/event_type.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Sub Event Type Data

In [ ]:
copy_sql = """
           COPY acled.sub_event_type FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/inputs/sub_event_type.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Actor Data

In [ ]:
copy_sql = """
           COPY acled.actor FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/outputs/actor.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Associated Actor Data

In [ ]:
copy_sql = """
           COPY acled.assoc_actor FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/outputs/assoc_actor.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Interaction Code Data

In [ ]:
copy_sql = """
           COPY acled.inter_code FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/inputs/inter_code.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Interactions Data

In [ ]:
copy_sql = """
           COPY acled.interaction FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/inputs/interaction.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Source Data

In [ ]:
copy_sql = """
           COPY acled.source FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/outputs/source.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### Source Scale Data

In [ ]:
copy_sql = """
           COPY acled.source_scale FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/outputs/source_scale.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

### ACLED Data

In [ ]:
copy_sql = """
           COPY acled.acled FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
with open('../ACLED/outputs/acled.csv', 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    conn.commit()

## Remove CSV Output Files

In [ ]:
import os

In [ ]:
# We will remove the output file now that its data is in the database
os.remove('../ACLED/outputs/acled.csv')
os.remove('../ACLED/outputs/actor.csv')
os.remove('../ACLED/outputs/assoc_actor.csv')
os.remove('../ACLED/outputs/source_scale.csv')
os.remove('../ACLED/outputs/source.csv')

### Disconnect from Database

In [18]:
try: 
    # close the communication with the PostgreSQL
    cur.close()
    
    # close the connection with the database
    conn.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
        